<a href="https://colab.research.google.com/github/Shakilkhan24/Playground_DL/blob/main/CNN_IMPLEMENTATIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAvgPool2D,Activation,DepthwiseConv2D,AvgPool2D,Dropout,Concatenate,Flatten,Input,Conv2D,ReLU,Dense,BatchNormalization,MaxPool2D


ALEXNET::

In [11]:


def alexNet(input_shape,n_classes):
  input=Input(input_shape)
  x=Conv2D(96,kernel_size=(11,11),strides=(4,4),padding='same',activation='relu')(input)
  x=BatchNormalization()(x)
  x=MaxPool2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

  x=Conv2D(256,5,padding='same',activation='relu')(x)
  x=BatchNormalization()(x)
  x=MaxPool2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

  x=Conv2D(384,3,padding='same',activation='relu')(x)
  x=Conv2D(384,3,padding='same',activation='relu')(x)
  x=Conv2D(256,3,padding='same',activation='relu')(x)
  x=BatchNormalization()(x)
  x=MaxPool2D(pool_size=(3,3),strides=2)

  x=Flatten(4096,activation='relu')(x)
  x=Dense(4096,activation='relu')(x)
  output=Dense(n_classes,activation='softmax')(x)

  model=Model(input,output)
  return model


VGG

In [13]:
def vgg16(input_shape,n_classes):
  input=Input(input_shape)
  x=Conv2D(64,3,padding='same',activation='relu')(input)
  x=Conv2D(64,3,padding='same',activation='relu')(x)
  x=MaxPool2D(2,strides=2)(x)

  x=Conv2D(128,3,padding='same',activation='relu')(x)
  x=Conv2D(128,3,padding='same',activation='relu')(x)
  x=MaxPool2D(2,strides=2)(x)

  x=Conv2D(512,3,padding='same',activation='relu')(x)
  x=Conv2D(512,3,padding='same',activation='relu')(x)
  x=Conv2D(512,3,padding='same',activation='relu')(x)
  x=MaxPool2D(2,strides=2)(x)

  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)


  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)

  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model




GoogleNet

In [19]:
def googlenet(input_shape, n_classes):

  def inception_block(x, f):
    t1 = Conv2D(f[0], 1, activation='relu')(x)

    t2 = Conv2D(f[1], 1, activation='relu')(x)
    t2 = Conv2D(f[2], 3, padding='same', activation='relu')(t2)

    t3 = Conv2D(f[3], 1, activation='relu')(x)
    t3 = Conv2D(f[4], 5, padding='same', activation='relu')(t3)

    t4 = MaxPool2D(3, 1, padding='same')(x)
    t4 = Conv2D(f[5], 1, activation='relu')(t4)

    output = Concatenate()([t1, t2, t3, t4])
    return output


  input = Input(input_shape)

  x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = Conv2D(64, 1, activation='relu')(x)
  x = Conv2D(192, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(3, strides=2)(x)

  x = inception_block(x, [64, 96, 128, 16, 32, 32])
  x = inception_block(x, [128, 128, 192, 32, 96, 64])
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = inception_block(x, [192, 96, 208, 16, 48, 64])
  x = inception_block(x, [160, 112, 224, 24, 64, 64])
  x = inception_block(x, [128, 128, 256, 24, 64, 64])
  x = inception_block(x, [112, 144, 288, 32, 64, 64])
  x = inception_block(x, [256, 160, 320, 32, 128, 128])
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = inception_block(x, [256, 160, 320, 32, 128, 128])
  x = inception_block(x, [384, 192, 384, 48, 128, 128])

  x = AvgPool2D(7, strides=1)(x)
  x = Dropout(0.4)(x)

  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model



MobileNet

In [20]:
def mobilenet(input_shape, n_classes):

  def mobilenet_block(x, f, s=1):
    x = DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(f, 1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x


  input = Input(input_shape)

  x = Conv2D(32, 3, strides=2, padding='same')(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  x = mobilenet_block(x, 64)
  x = mobilenet_block(x, 128, 2)
  x = mobilenet_block(x, 128)

  x = mobilenet_block(x, 256, 2)
  x = mobilenet_block(x, 256)

  x = mobilenet_block(x, 512, 2)
  for _ in range(5):
    x = mobilenet_block(x, 512)

  x = mobilenet_block(x, 1024, 2)
  x = mobilenet_block(x, 1024)

  x = GlobalAvgPool2D()(x)

  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model

SqueezeNet

In [22]:
def squeezenet(input_shape, n_classes):

  def fire(x, fs, fe):
    s = Conv2D(fs, 1, activation='relu')(x)
    e1 = Conv2D(fe, 1, activation='relu')(s)
    e3 = Conv2D(fe, 3, padding='same', activation='relu')(s)
    output = Concatenate()([e1, e3])
    return output


  input = Input(input_shape)

  x = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = fire(x, 16, 64)
  x = fire(x, 16, 64)
  x = fire(x, 32, 128)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = fire(x, 32, 128)
  x = fire(x, 48, 192)
  x = fire(x, 48, 192)
  x = fire(x, 64, 256)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = fire(x, 64, 256)
  x = Conv2D(n_classes, 1)(x)
  x = GlobalAvgPool2D()(x)

  output = Activation('softmax')(x)

  model = Model(input, output)
  return model

ResNet

In [23]:
def resnet(input_shape, n_classes):

  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x


  def identity_block(tensor, f):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)

    x = Add()([x, tensor])
    output = ReLU()(x)
    return output


  def conv_block(tensor, f, s):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3, s)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(4*f, 1, strides=s)(tensor)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    output = ReLU()(x)
    return output


  def resnet_block(x, f, r, s=2):
    x = conv_block(x, f, s)
    for _ in range(r-1):
      x = identity_block(x, f)
    return x


  input = Input(input_shape)

  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = resnet_block(x, 64, 3, 1)
  x = resnet_block(x, 128, 4)
  x = resnet_block(x, 256, 6)
  x = resnet_block(x, 512, 3)

  x = GlobalAvgPool2D()(x)

  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model

DenseNet

In [24]:
def densenet(img_shape, n_classes, f=32):
  repetitions = 6, 12, 24, 16

  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x


  def dense_block(tensor, r):
    for _ in range(r):
      x = bn_rl_conv(tensor, 4*f)
      x = bn_rl_conv(x, f, 3)
      tensor = Concatenate()([tensor, x])
    return tensor


  def transition_block(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x


  input = Input(img_shape)

  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  for r in repetitions:
    d = dense_block(x, r)
    x = transition_block(d)

  x = GlobalAvgPool2D()(d)

  output = Dense(n_classes, activation='softmax')(x)

  model = Model(input, output)
  return model